In [ ]:
#default_exp vision.utils

In [ ]:
#export
from fastai2.test import *
from fastai2.torch_basics import *
from fastai2.data.all import *

from fastai2.vision.core import *

In [ ]:
from nbdev.showdoc import *

# Vision utils

> Some utils function to quickly download a bunch of images, check them and pre-resize them

In [ ]:
#export
def _download_image_inner(dest, inp, timeout=4):
    i,url = inp
    suffix = re.findall(r'\.\w+?(?=(?:\?|$))', url)
    suffix = suffix[0] if len(suffix)>0  else '.jpg'
    try: download_url(url, dest/f"{i:08d}{suffix}", overwrite=True, show_progress=False, timeout=timeout)
    except Exception as e: f"Couldn't download {url}."

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    url = "https://www.fast.ai/images/jh-head"
    _download_image_inner(d, (125,url))
    assert (d/'00000125.jpg').is_file()

In [ ]:
#export
def download_images(url_file, dest, max_pics=1000, n_workers=8, timeout=4):
    "Download images listed in text file `url_file` to path `dest`, at most `max_pics`"
    urls = url_file.read().strip().split("\n")[:max_pics]
    dest = Path(dest)
    dest.mkdir(exist_ok=True)
    parallel(partial(_download_image_inner, dest, timeout=timeout), list(enumerate(urls)), n_workers=n_workers)

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    url_file = (d/'urls.txt')
    url_file.write("\n".join([f"https://www.fast.ai/images/{n}" for n in "jh-head thomas.JPG sg-head".split()]))
    download_images(url_file, d)
    for i in [0,2]: assert (d/f'0000000{i}.jpg').is_file()
    assert (d/f'00000001.JPG').is_file()

In [ ]:
#export
def resize_to(img, targ_sz, use_min=False):
    "Size to resize to, to hit `targ_sz` at same aspect ratio, in PIL coords (i.e w*h)"
    w,h = img.size
    min_sz = (min if use_min else max)(w,h)
    ratio = targ_sz/min_sz
    return int(w*ratio),int(h*ratio)

In [ ]:
class _FakeImg():
    def __init__(self, size): self.size=size
img = _FakeImg((200,500))

test_eq(resize_to(img, 400), [160,400])
test_eq(resize_to(img, 400, use_min=True), [400,1000])

In [ ]:
#export
@delegates(Image.Image.save)
def verify_image(file, delete=True, max_size=None, dest=None, n_channels=3, interp=Image.BILINEAR, ext=None, img_format=None, resume=False, **kwargs):
    "Check if the image in `file` exists, maybe resize it and copy it in `dest`."
    try:
        # deal with partially broken images as indicated by PIL warnings
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try: Image.open(file)
            except Warning as w:
                if "Possibly corrupt EXIF data" in str(w):
                    if delete: # green light to modify files
                        print(f"{file}: Removing corrupt EXIF data")
                        warnings.simplefilter("ignore")
                        PIL.Image.open(file).save(file)
                    else: print(f"{file}: Corrupt EXIF data, pass `delete=True` to remove it")
                else: warnings.warn(w)

        img = Image.open(file)
        imgarr = np.array(img)
        img_channels = 1 if len(imgarr.shape) == 2 else imgarr.shape[2]
        if (max_size is not None and (img.height > max_size or img.width > max_size)) or img_channels != n_channels:
            assert dest is not None, "You should provide `dest` Path to save resized image"
            dest_fname = Path(dest)/file.name
            if ext is not None: dest_fname=dest_fname.with_suffix(ext)
            if resume and os.path.isfile(dest_fname): return
            if max_size is not None:
                new_sz = resize_to(img, max_size)
                img = img.resize(new_sz, resample=interp)
            if n_channels == 3: img = img.convert("RGB")
            img.save(dest_fname, img_format, **kwargs)
    except Exception as e:
        print(f'{e}')
        if delete: file.unlink()

In [ ]:
#export
@delegates(verify_image)
def verify_images(path, max_workers=4, recurse=False, dest='.', **kwargs):
    "Check if the images in `path` aren't broken, maybe resize them and copy it in `dest`."
    path = Path(path)
    dest = path/Path(dest)
    os.makedirs(dest, exist_ok=True)
    files = get_image_files(path, recurse=recurse)
    func = partial(verify_image, dest=dest, **kwargs)
    parallel(func, files, max_workers=max_workers)

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_core_foundation.ipynb.
Converted 01a_core_utils.ipynb.
Converted 01b_core_dispatch.ipynb.
Converted 01c_core_transform.ipynb.
Converted 02_core_script.ipynb.
Converted 03_torchcore.ipynb.
Converted 03a_layers.ipynb.
Converted 04_data_load.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_transforms.ipynb.
Converted 07_data_block.ipynb.
Converted 08_vision_core.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_vision_data.ipynb.
Converted 09b_vision_utils.ipynb.
Converted 10_pets_tutorial.ipynb.
Converted 11_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 14a_callback_data.ipynb.
Converted 15_callback_hook.ipynb.
Converted 15a_vision_models_unet.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_interpret.ipynb.
C